In [224]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import tensorflow as tf
import math

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

In [ ]:
xy = np.loadtxt('ML_data2.csv',delimiter=',',dtype=np.float32)

scaler = MinMaxScaler()
xy2 = xy

x_data = xy2[:,1:-1]
y_data = xy2[:,[-1]]/1000000000

scaler.fit(x_data)
x_data = scaler.transform(x_data)

x_train, x_test, y_train, y_test = train_test_split(x_data,y_data,random_state=123)

x = tf.placeholder(tf.float32,shape=[None,10],name='x')
y = tf.placeholder(tf.float32,shape=[None,1],name='y')

w = tf.Variable(tf.random_normal([10,1]),dtype=tf.float32,name='w')
b = tf.Variable(tf.random_normal([1]),dtype=tf.float32,name='b')

hypothesis = tf.matmul(x,w) + b

cost = tf.reduce_mean(tf.square(hypothesis - y),name='cost')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0004,name='optimizer')
train = optimizer.minimize(cost)

#saver = tf.compat.v1.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer())


#coord = tf.train.Coordinator()
#threads = tf.train.start_queue_runners(sess=sess, coord=coord)

feed_train = {x:x_train,y:y_train}
feed_test = {x:x_test,y:y_test}
for step in range(500001):
    c2,h2,_ = sess.run([cost,hypothesis,train],
                    feed_dict=feed_train)
    
    if step%1000 == 0:
        print(step,'cost:',c2,'\npredict:',h2)
        
        
        
#saver = tf.train.Saver()
#save_path = saver.save(sess, 'model/auction/lys_auction_model_1_0.ckpt')
#joblib.dump(scaler, 'model/auction/a_scaler.gz')

In [239]:
# 매물추천 프로그램 함수화
# 입력 값에 따라서 데이터 프레임 개인의 고유 데이터 프레임 생성


def type_items(type_):
    
    box = []
    cnt = 0
    cnt_err = 0
    for step in range(len(recommend_df)):
        if recommend_df['recommend'][step] in type_:
            if recommend_df['y1'][step] < 0 or recommend_df['diff_high'][step] < 0 :
                cnt_err += 1
            
            else:
                cnt +=1
                box.append(recommend_df['y1'][step])
                print(recommend_df.loc[step,['addr_name','brand_name']],'예측:','예상낙찰가:{}억 ,예상차익:{}억, 투자가능도:{}, 예상경쟁률:{}'.format(round(recommend_df['y1'][step]/100000000,3),round(recommend_df['diff_high'][step]/100000000,3),recommend_df['pred_budget'][step],recommend_df['commpetition'][step]))
                print('-'*120)
        
    if len(box) == 0:
        print('확인된 매물이 없습니다.\n\n\n') 
        
    else:
        print('확인된 매물:{} 개\n'.format(cnt))
        
        
        
def soul_pull(soul):
    if soul == '2':
        
            return False
            
    elif soul == '1':
        for step in range(len(recommend_df)):
            if recommend_df['soul'][step] in '영끌추천':
                print('예상 수이익:',recommend_df['diff_high'][step],'영끌추천')
                #print(recommend[['hammer','y1','y2']])    
        print('최저금리로 5분대출 전화한통일면 OK \n070-0000-0000 언제나 열려있습니다. \n가족처럼 모시겠습니다.\n\n\n\n\n\n')
        
        print('종료하시려면 2번을 누르세요. \n처음으로 돌아가시려면 아무키나 누르세요.')
        end_ = input('종료하시겠습니까?')

        if end_ in '2':
            return 0
        
        

        
        
def auction(df):
    import joblib
    import tensorflow as tf
    
    gu_cd = pd.read_csv("model/auction/a_gu_cd.csv")
    dong_cd = pd.read_csv("model/auction/a_dong_cd.csv")
    
    # 구, 동 이름 숫자 코딩
    for i in df.index:
        df.loc[i,"gu_num"] = int(gu_cd.loc[gu_cd.gu==df.loc[i,"gu"],"gu_num"])
        df.loc[i,"dong_num"] = int(dong_cd.loc[dong_cd.dong==df.loc[i,"dong"],"dong_num"])

    # 필요한 열 뽑기
    x_data = df[['lowest','price','low_per','gu_num','dong_num','date_ym','floor','build_area','comment_val','case_year']].dropna()
    
    # 스케일링
    scaler = joblib.load("model/auction/a_scaler.gz")
    x_data = scaler.transform(x_data)

    # 모델 불러오기  
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    saver = tf.train.import_meta_graph("model/auction/lys_auction_model_1_0.ckpt.meta")
    saver.restore(sess,"model/auction/lys_auction_model_1_0.ckpt")  
    graph = tf.get_default_graph()
    
    x = graph.get_tensor_by_name("x:0")
    
    w = graph.get_tensor_by_name("w:0")
    b = graph.get_tensor_by_name("b:0")
    
    hypot = tf.matmul(x,w)+b
    cost = graph.get_tensor_by_name("cost:0")
#     optimizer = graph.get_operation_by_name("optimizer:0")
    
    print(hypot)
    
    # 낙찰가 예측
    h = sess.run(hypot, feed_dict={x:x_data})

    return h.flatten() 



def transaction(df):
    from tensorflow.keras.models import load_model
    from sklearn.preprocessing import MinMaxScaler
    import joblib
    
    gu_cd = pd.read_csv("model/transaction/t_gu_cd.csv")
    dong_cd = pd.read_csv("model/transaction/t_dong_cd.csv")
#    brand_cd = pd.read_csv("model/transaction/t_brand_cd.csv")
    
    # 구, 동, 아파트 이름 숫자 코딩, 사용연수 추가 -> 아파트이름, 사용연수 제외
    for i in df.index:
        try:
            df.loc[i,"구_int"] = int(gu_cd.loc[gu_cd.구==df.loc[i,"gu"],"구_int"])
            df.loc[i,"동_int"] = int(dong_cd.loc[dong_cd.동==df.loc[i,"dong"],"동_int"])
#            df.loc[i,"brand_int"] = int(brand_cd.loc[(brand_cd.구==df.loc[i,"gu"])&(brand_cd.동==df.loc[i,"dong"])&(brand_cd.brand_name==df.loc[i,"brand_name"]),"brand_name_int"])
#            df.loc[i,"years_use"] = int(brand_cd.loc[(brand_cd.구==df.loc[i,"gu"])&(brand_cd.동==df.loc[i,"dong"])&(brand_cd.brand_name==df.loc[i,"brand_name"]),"years_use"])
        
        except TypeError:
            pass
    
    # 계약년월 받아서 추가   
    while True:
        date_ym = input("예상 매매날짜(연,월)를 YYYYMM 형태로 기입하세요.")
        
        if len(date_ym) > 6:
            print("날짜를 연월(YYYYMM) 형태로 기입해주세요.(일 제외) 예) 202210")
            continue
            
        elif (int(date_ym[:4]) < 2100) & (int(date_ym[4:]) <= 12):
            date_ym = int(date_ym)
            break

        else:
            print("날짜를 확인해주세요.")
            continue
        
    df["계약년월"] = date_ym

    # 필요한 열 뽑기
    x_data = df[["계약년월","구_int","동_int","build_area","floor"]]
    
    #print(x_data)
    #x_data = x_data.dropna()
    
    # 모델 불러오기
    model = load_model("model/transaction/model5.h5")
#    print(mse, mae)
    
    # 스케일링
    scaler = joblib.load("model/transaction/t_scaler.gz")
    x_data = scaler.transform(x_data)
    
    # 실거래가 예측
    y = model.predict(x_data)

    return y.flatten()



def recommend_apt(data):
    import pandas as pd
    import numpy as np
    
    global recommend_df
    
    recommend_df = pd.DataFrame(data)
    
    recommend_df['y1'] = auction(recommend_df)*1000000000                     # 경매가 예측 함수 호출
    recommend_df['cost1'] = math.pow(0.011388925,2)*1000000000                # 예측가격 컬럼 생성
    
    auc_pred = recommend_df['y1']
    auc_cost = recommend_df['cost1']
    
    recommend_df['y2'] = transaction(df)*1000000000
    recommend_df['cost2'] = 0.02639*1000000000
    
    trans_pred = recommend_df['y2']
    trans_cost = recommend_df['cost2']
    
    formula1_p = (auc_pred + auc_cost)
    formula1_m = (auc_pred - auc_cost)
    
    try:
        
        b = input('자본금을 입력하세요:')  # 자본금 입력
        b = int(b)        
                
        recommend_df['b'] = b
        
        recommend_df['b_ratio_high'] = b / formula1_p                         # 자본금 대비 매물가격의 비율 컬럼 생성 
        recommend_df['b_ratio_low'] = b / formula1_m
       
        recommend_df['ham_per_high'] = recommend_df['price']/formula1_p*100   # 예상 낙찰가 비율 컬럼 생성
        recommend_df['ham_per'] = recommend_df['price']/auc_pred*100
        recommend_df['ham_per_low'] = recommend_df['price']/formula1_m*100
        
        recommend_df['diff_low'] = trans_pred - formula1_m                    # 낙찰가 대비 실거래가 차익 컬럼 생성
        recommend_df['diff_high'] = trans_pred - formula1_p
        
        
        # 여러 조건에 따른 경쟁도 측정 컬럼 생성
        for step in range(len(recommend_df)):
            if recommend_df['ham_per_high'][step] >= 100:
                recommend_df.loc[step,'commpetition'] = '치열'
                
            elif recommend_df['ham_per'][step] >= 100:
                recommend_df.loc[step,'commpetition'] = '경계'
                
            else:
                recommend_df.loc[step,'commpetition'] = 'cool'
        
        
        # 여러 조건에 따른 예상 예산안 컬럼 생성
        for step in range(len(recommend_df)):
            if recommend_df['b_ratio_high'][step] >= 100 and recommend_df['b_ratio_low'][step] >= 100:
                recommend_df.loc[step,'pred_budget'] = '적정'
                
            elif recommend_df['b_ratio_high'][step] >= 100 and recommend_df['b_ratio_low'][step] <= 100:
                recommend_df.loc[step,'pred_budget'] = '도전'
                    
            else:
                recommend_df.loc[step,'pred_budget'] = '도박'
                 
        
        
        # 여러 조건에 따른 추천 투자방향 컬럼 생성
        for step in range(len(recommend_df)):
            if recommend_df['pred_budget'][step] in '적정' and recommend_df['commpetition'][step] in 'cool':
                recommend_df.loc[step,'recommend'] = '1'
            
            elif recommend_df['pred_budget'][step] in '적정' and recommend_df['commpetition'][step] in '경계':
                recommend_df.loc[step,'recommend'] = '2'
            
            elif recommend_df['pred_budget'][step] in '적정' and recommend_df['commpetition'][step] in '치열':
                recommend_df.loc[step,'recommend'] = '3'
                
            elif recommend_df['pred_budget'][step] in '도전' and recommend_df['commpetition'][step] in 'cool':
                recommend_df.loc[step,'recommend'] = '2'
            
            elif recommend_df['pred_budget'][step] in '도전' and recommend_df['commpetition'][step] in '경계':
                recommend_df.loc[step,'recommend'] = '3'
            
            elif recommend_df['pred_budget'][step] in '도전' and recommend_df['commpetition'][step] in '치열':
                recommend_df.loc[step,'recommend'] = '4'
            
            elif recommend_df['pred_budget'][step] in '도박' and recommend_df['commpetition'][step] in 'cool':
                recommend_df.loc[step,'recommend'] = '3'
                    
            elif recommend_df['pred_budget'][step] in '도박' and recommend_df['commpetition'][step] in '경계':
                recommend_df.loc[step,'recommend'] = '4'
            
            elif recommend_df['pred_budget'][step] in '도박' and recommend_df['commpetition'][step] in '치열':
                recommend_df.loc[step,'recommend'] = '5'
            
            else:
                recommend_df.loc[step,'recommend'] = '실패'
            
            
                
        for step in range(len(recommend_df)):
            if recommend_df['diff_high'][step] >= 500000000 and recommend_df['pred_budget'][step] in '도박':
                 recommend_df.loc[step,'soul'] = '영끌추천'
                
            else:
                 recommend_df.loc[step,'soul'] = '영끌비추천'
                          
            
    except TypeError:
        pass
    
           
    while True:
        print('번호나 이름을 입력하시오. \n1.완벽주의 투자자 \n2.신중하 투자자 \n3.적극적인 투자자 \n4.모험심 많은 투자자 \n5.공격적인 투자자')
        Investment_type = input('투자타입을 설정하세요:')
        print('\n\n\n')
        if Investment_type in '1':
            type_items(Investment_type)
            
            print('확인을 원하시면 1번을 누르세요. \n뒤로 가시려면 아무키나 누르세요. \n종료를 원하시면 2번을 누르세요.')
            soul = input('적정가격 이외의 고수익 매물을 확인하시겠습니까?')
            if soul in '2':
                break
                
            # soul_pull함수 호출
            soul_pull(soul)
            if soul_pull in '0':
                break
            
        elif Investment_type in '2':
            type_items(Investment_type)
            
            print('확인을 원하시면 1번을 누르세요. \n뒤로 가시려면 아무키나 누르세요. \n종료를 원하시면 2번을 누르세요.')    
            soul = input('적정가격 이외의 고수익 매물을 확인하시겠습니까?')
            
            if soul in '2':
                break
                
            # soul_pull함수 호출
            soul_pull(soul)
            if soul_pull in '0':
                break
            
        elif Investment_type in '3':
            type_items(Investment_type)
            
            print('확인을 원하시면 1번을 누르세요. \n뒤로 가시려면 아무키나 누르세요. \n종료를 원하시면 2번을 누르세요.')        
            soul = input('적정가격 이외의 고수익 매물을 확인하시겠습니까?')
             
            if soul in '2':
                break
                
            # soul_pull함수 호출
            soul_pull(soul)
            if soul_pull in '0':
                break
            
        elif Investment_type in '4':
            type_items(Investment_type)
            
            print('확인을 원하시면 1번을 누르세요. \n뒤로 가시려면 아무키나 누르세요. \n종료를 원하시면 2번을 누르세요.')        
            soul = input('적정가격 이외의 고수익 매물을 확인하시겠습니까?')
            
            if soul in '2':
                break
                
            # soul_pull함수 호출
            soul_pull(soul)
            if soul_pull in '0':
                break
            
        elif Investment_type in '5':
            type_items(Investment_type)
            
            print('확인을 원하시면 1번을 누르세요. \n뒤로 가시려면 아무키나 누르세요. \n종료를 원하시면 2번을 누르세요.')        
            soul = input('적정가격 이외의 고수익 매물을 확인하시겠습니까?')
            
            if soul in '2':
                break
                
            # soul_pull함수 호출
            soul_pull(soul)
            if soul_pull in '0':
                break
            
        else:
            print('잘못 입력되었습니다.\n\n\n')
                
                
 
        

In [ ]:
df = pd.read_csv('../data/auction/auction_pjs3.csv')
recommend_apt(df)

INFO:tensorflow:Restoring parameters from model/auction/lys_auction_model_1_0.ckpt
Tensor("add_123:0", shape=(?, 1), dtype=float32)
예상 매매날짜(연,월)를 YYYYMM 형태로 기입하세요.202508
자본금을 입력하세요:1000000000
번호나 이름을 입력하시오. 
1.완벽주의 투자자 
2.신중하 투자자 
3.적극적인 투자자 
4.모험심 많은 투자자 
5.공격적인 투자자
투자타입을 설정하세요:3




addr_name     서울특별시 성동구 홍익동 53
brand_name          청계대주파크빌아파트
Name: 1, dtype: object 예측: 예상낙찰가:4.859억 ,예상차익:nan억, 투자가능도:도박, 예상경쟁률:cool
------------------------------------------------------------------------------------------------------------------------
addr_name     서울특별시 서대문구 홍제동 470
brand_name                 제일아파트
Name: 4, dtype: object 예측: 예상낙찰가:5.802억 ,예상차익:6.453억, 투자가능도:도박, 예상경쟁률:cool
------------------------------------------------------------------------------------------------------------------------
addr_name     서울특별시 은평구 구산동 224
brand_name            경남아너스빌아파트
Name: 5, dtype: object 예측: 예상낙찰가:5.088억 ,예상차익:4.776억, 투자가능도:도박, 예상경쟁률:cool
---------------------------------------------------------

addr_name     서울특별시 서대문구 대현동 101-7
brand_name                   혜우아파트
Name: 114, dtype: object 예측: 예상낙찰가:5.805억 ,예상차익:6.489억, 투자가능도:도박, 예상경쟁률:cool
------------------------------------------------------------------------------------------------------------------------
addr_name     서울특별시 광진구 자양동 813
brand_name         광진하우스토리한강아파트
Name: 115, dtype: object 예측: 예상낙찰가:15.857억 ,예상차익:1.119억, 투자가능도:도박, 예상경쟁률:cool
------------------------------------------------------------------------------------------------------------------------
addr_name     서울특별시 송파구 장지동 849
brand_name         송파파인타운3단지아파트
Name: 116, dtype: object 예측: 예상낙찰가:11.7억 ,예상차익:5.301억, 투자가능도:도박, 예상경쟁률:cool
------------------------------------------------------------------------------------------------------------------------
addr_name     서울특별시 강동구 암사동 501-4
brand_name             암사동양덱스빌아파트
Name: 119, dtype: object 예측: 예상낙찰가:7.123억 ,예상차익:nan억, 투자가능도:도박, 예상경쟁률:cool
-----------------------------------------------------------------

Name: 322, dtype: object 예측: 예상낙찰가:5.13억 ,예상차익:7.182억, 투자가능도:도박, 예상경쟁률:cool
------------------------------------------------------------------------------------------------------------------------
addr_name     서울특별시 서초구 방배동 482-4
brand_name            청광아트빌14차아파트
Name: 323, dtype: object 예측: 예상낙찰가:16.314억 ,예상차익:1.86억, 투자가능도:도박, 예상경쟁률:cool
------------------------------------------------------------------------------------------------------------------------
addr_name     서울특별시 서초구 방배동 482-4
brand_name            청광아트빌14차아파트
Name: 324, dtype: object 예측: 예상낙찰가:16.433억 ,예상차익:1.352억, 투자가능도:도박, 예상경쟁률:cool
------------------------------------------------------------------------------------------------------------------------
addr_name     서울특별시 양천구 목동 950
brand_name          목동이편한세상아파트
Name: 327, dtype: object 예측: 예상낙찰가:13.699억 ,예상차익:1.154억, 투자가능도:도박, 예상경쟁률:cool
------------------------------------------------------------------------------------------------------------------------
addr_name

Name: 477, dtype: object 예측: 예상낙찰가:4.248억 ,예상차익:12.886억, 투자가능도:도박, 예상경쟁률:cool
------------------------------------------------------------------------------------------------------------------------
addr_name     서울특별시 노원구 상계동 1285
brand_name               동아불암아파트
Name: 479, dtype: object 예측: 예상낙찰가:3.466억 ,예상차익:8.247억, 투자가능도:도박, 예상경쟁률:cool
------------------------------------------------------------------------------------------------------------------------
addr_name     서울특별시 도봉구 창동 731-54
brand_name                 새로움아파트
Name: 480, dtype: object 예측: 예상낙찰가:3.8억 ,예상차익:8.886억, 투자가능도:도박, 예상경쟁률:cool
------------------------------------------------------------------------------------------------------------------------
addr_name     서울특별시 서대문구 충정로3가 477
brand_name                충림예하임아파트
Name: 487, dtype: object 예측: 예상낙찰가:6.748억 ,예상차익:nan억, 투자가능도:도박, 예상경쟁률:cool
------------------------------------------------------------------------------------------------------------------------
addr_na

Name: 696, dtype: object 예측: 예상낙찰가:5.577억 ,예상차익:7.011억, 투자가능도:도박, 예상경쟁률:cool
------------------------------------------------------------------------------------------------------------------------
addr_name     서울특별시 서대문구 홍제동 459
brand_name              홍제원현대아파트
Name: 699, dtype: object 예측: 예상낙찰가:7.001억 ,예상차익:5.589억, 투자가능도:도박, 예상경쟁률:cool
------------------------------------------------------------------------------------------------------------------------
addr_name     서울특별시 성동구 옥수동 4
brand_name            옥수현대아파트
Name: 703, dtype: object 예측: 예상낙찰가:4.719억 ,예상차익:nan억, 투자가능도:도박, 예상경쟁률:cool
------------------------------------------------------------------------------------------------------------------------
addr_name     서울특별시 영등포구 대림동 785
brand_name               현대2차아파트
Name: 704, dtype: object 예측: 예상낙찰가:5.672억 ,예상차익:9.649억, 투자가능도:도박, 예상경쟁률:cool
------------------------------------------------------------------------------------------------------------------------
addr_name     서울특별

Name: 823, dtype: object 예측: 예상낙찰가:6.248억 ,예상차익:4.765억, 투자가능도:도박, 예상경쟁률:cool
------------------------------------------------------------------------------------------------------------------------
addr_name     서울특별시 은평구 응암동 739
brand_name              응암금호아파트
Name: 824, dtype: object 예측: 예상낙찰가:4.888억 ,예상차익:4.663억, 투자가능도:도박, 예상경쟁률:cool
------------------------------------------------------------------------------------------------------------------------
addr_name     서울특별시 용산구 효창동 5-145
brand_name                효창한신아파트
Name: 826, dtype: object 예측: 예상낙찰가:6.386억 ,예상차익:6.144억, 투자가능도:도박, 예상경쟁률:cool
------------------------------------------------------------------------------------------------------------------------
addr_name     서울특별시 도봉구 쌍문동 59
brand_name         한양2,3,4차아파트
Name: 832, dtype: object 예측: 예상낙찰가:1.838억 ,예상차익:2.49억, 투자가능도:도박, 예상경쟁률:cool
------------------------------------------------------------------------------------------------------------------------
addr_name     서

Name: 1036, dtype: object 예측: 예상낙찰가:1.907억 ,예상차익:0.542억, 투자가능도:도박, 예상경쟁률:cool
------------------------------------------------------------------------------------------------------------------------
addr_name     서울특별시 성북구  성북로4길 52
brand_name                한신한진아파트
Name: 1038, dtype: object 예측: 예상낙찰가:6.27억 ,예상차익:10.272억, 투자가능도:도박, 예상경쟁률:cool
------------------------------------------------------------------------------------------------------------------------
addr_name     서울특별시 구로구  개봉로20길 6
brand_name               현대1단지아파트
Name: 1039, dtype: object 예측: 예상낙찰가:6.555억 ,예상차익:1.963억, 투자가능도:도박, 예상경쟁률:cool
------------------------------------------------------------------------------------------------------------------------
addr_name     서울특별시 영등포구  영등포로28길 5
brand_name                 당산코오롱아파트
Name: 1040, dtype: object 예측: 예상낙찰가:5.023억 ,예상차익:2.966억, 투자가능도:도박, 예상경쟁률:cool
---------------------------------------------------------------------------------------------------------------------